In [1]:
import pandas as pd
import numpy as np
import pickle
import treecorr
from tqdm import tqdm_notebook as tqdm

In [2]:
df_all=[None]*4

In [3]:
for i in range(4):
    df_all[i]=pd.read_pickle('/home/cz136/project/balrog/data/mcal{}_df_all.pkl'.format(i+2))

In [4]:
df_riz = pd.read_pickle('/home/cz136/project/balrog/data/mcal{}_df_all.pkl'.format(3)) 

In [5]:
# keys = df_riz.columns[40:]
keys=['ebv_planck13']

In [6]:
true_df=pd.read_pickle("/home/cz136/project/balrog/data/true_df.pkl")
mcal_list = [df_all]
df_riz_list= [df_riz]

In [7]:
def add_tilename(df_list,true):
    for i in range(4):
        df_list[i]=pd.concat([df_list[i],true_df[['meas_tilename']]],axis=1,join='inner')

In [8]:
for df_list in mcal_list:
    add_tilename(df_list,true_df)

In [9]:
df_riz=pd.concat([df_riz,true_df[['meas_tilename']]],axis=1,join='inner')

In [10]:
def get_ke(catalog,key):
    delta_g1=catalog['e_1'].to_numpy()
    delta_g2=catalog['e_2'].to_numpy()
    k=catalog[key].to_numpy()
    ra=catalog['ra'].to_numpy()
    dec=catalog['dec'].to_numpy()
    
    kcat= treecorr.Catalog( k=k,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    gcat = treecorr.Catalog( g1=delta_g1, g2=delta_g2,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    kg = treecorr.KGCorrelation(nbins=20, min_sep=2.5, max_sep=250, sep_units='arcmin', bin_slop=0.2)
    kg.process_cross(kcat,gcat)
    return(kg)

In [11]:
def get_ke_true(catalog,key):
    delta_g1=catalog['true_cm_g_1'].to_numpy()
    delta_g2=catalog['true_cm_g_2'].to_numpy()
    k=catalog[key].to_numpy()
    ra=catalog['ra'].to_numpy()
    dec=catalog['dec'].to_numpy()
    
    kcat= treecorr.Catalog( k=k,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    gcat = treecorr.Catalog( g1=delta_g1, g2=delta_g2,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    kg = treecorr.KGCorrelation(nbins=20, min_sep=2.5, max_sep=250, sep_units='arcmin', bin_slop=0.2)
    kg.process_cross(kcat,gcat)
    return(kg)

In [12]:
def get_ke_true_cross(catalog1,catalog2,key):
    delta_g1=np.concatenate((catalog1['true_cm_g_1'].to_numpy(),catalog2['true_cm_g_1'].to_numpy()))
    delta_g2=np.concatenate((catalog1['true_cm_g_2'].to_numpy(),catalog2['true_cm_g_2'].to_numpy()))
    k=np.concatenate((catalog1[key].to_numpy(),catalog2[key].to_numpy()))
    ra=np.concatenate((catalog1['ra'].to_numpy(),catalog2['ra'].to_numpy()))
    dec=np.concatenate((catalog1['dec'].to_numpy(),catalog2['dec'].to_numpy()))
    
    kcat= treecorr.Catalog( k=k,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    gcat = treecorr.Catalog( g1=delta_g1, g2=delta_g2,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    kg = treecorr.KGCorrelation(nbins=20, min_sep=2.5, max_sep=250, sep_units='arcmin', bin_slop=0.2)
    kg.process_cross(kcat,gcat)
    return(kg)

In [13]:
tile_list=df_riz['meas_tilename'].unique()
        
def jk_resample(df,key):
    jk_ke={}
    jk_ke_true={}
    for i,tile in tqdm(enumerate(tile_list)):
        jk_sample=df[(df['meas_tilename']==tile)]
        jk_ke[tile]=get_ke(jk_sample,key)
        jk_ke_true[tile]=get_ke_true(jk_sample,key)
    return(jk_ke,jk_ke_true)

In [15]:
ind_samples={}
for key in keys:
    ind_samples[key]=jk_resample(df_riz,key)

KeyboardInterrupt: 

In [ ]:
def get_corr_matrix(df,key):
    n = len(tile_list)
    corr_matrix= [ [ None for i in range(n) ] for j in range(n) ] 
    corr_matrix_true=[ [ None for i in range(n) ] for j in range(n) ] 
    for i,tile_i in tqdm(enumerate(tile_list)):
        for j,tile_j in enumerate(tile_list):
            i_catalog=df[df['meas_tilename']==tile_i]
            j_catalog=df[df['meas_tilename']==tile_j]
            if i==j:
                corr_matrix[i][j]=get_ke(i_catalog,key)
                corr_matrix_true[i][j]=get_ke_true(i_catalog,key)
            if i!=j:
                corr_matrix[i][j]=get_ke_cross(i_catalog,j_catalog,key)
                corr_matrix_true[i][j]=get_ke_true_cross(i_catalog,j_catalog,key)
    return(corr_matrix,corr_matrix_true)

In [ ]:
for key in keys:
    get_corr_matrix(df_riz,key)

In [147]:
jk_samples={}


for key in keys:
    jk_samples[key]={}
    long_samples=ind_samples[key][0]
    long_samples_true=ind_samples[key][1]

    for tile in tile_list:
        xi = np.sum([kg.xi for tile_ind,kg in long_samples.items() if tile_ind != tile],axis=0) / np.sum([kg.npairs for tile_ind,kg in long_samples.items() if tile_ind!=tile],axis=0)
        xi_true = np.sum([kg.xi for tile_ind,kg in long_samples_true.items() if tile_ind!=tile], axis=0) / np.sum([kg.npairs for tile_ind,kg in long_samples_true.items() if tile_ind!=tile],axis=0)    
        jk_samples[key][tile]=(xi,xi_true)

/home/cz136/anaconda3/envs/balrog/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
/home/cz136/anaconda3/envs/balrog/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [161]:
def get_cov(jk_samples_for_key):
    
    n=len(jk_samples_for_key.values())
    xi_array=np.array([pair[0] for pair in jk_samples_for_key.values()])
    xi_true_array=np.array([pair[1] for pair in jk_samples_for_key.values()])
    diff_array=xi_array-xi_true_array
    emp_average=1/n*np.sum(diff_array,axis=0)
    nd = len(diff_array[0])
    cov=np.empty([nd,nd])
    for row in range(nd):
        for column in range(nd):
            cov[row][column]=\
            (n-1)/n * np.sum((diff_array[:,row]-emp_average[row])*(diff_array[:,column]-emp_average[column]))
    return (cov)

In [ ]:
list(jk_samples['ebv_planck13'].values())

In [155]:
len(jk_samples['ebv_planck13'].values())

995

In [158]:
def get_chisqr(covariance,y_value):
    return(np.dot(np.dot(y_value,np.linalg.inv(covariance)),y_value))

def get_chisqr_for_key(key):

    cov = get_cov(jk_samples[key])
    
    all_ke=get_ke(df_riz,key)
    all_ke_true=get_ke_true(df_riz,key)
    
    all_ke.finalize(0,0)
    all_ke_true.finalize(0,0)
    
    y_value=all_ke.xi-all_ke_true.xi
    
    chi_sqr=get_chisqr(cov,y_value)
    
    return(chi_sqr)

In [166]:
import matplotlib.pyplot as plt

def plot_error(key):

    error=np.sqrt(get_cov(jk_samples[key]).diagonal())

    all_ke=get_ke(df_riz,key)
    all_ke_true=get_ke_true(df_riz,key)
    
    all_ke.finalize(0,0)
    all_ke_true.finalize(0,0)
    
    x_val=np.exp(all_ke.meanlogr)
    y_val = all_ke_true.xi-all_ke.xi
        
    fig, ax = plt.subplots()

    ax.set_xscale('log')
    ax.set_yscale('log')

    ax.errorbar(x=x_val,y=y_val,yerr=error,ls='none')
    
    ax.plot(x_val,y_val)
    
    for tile in jk_samples[key].keys():
        ax.scatter(x_val,jk_samples[key][tile][1]-jk_samples[key][tile][0])

    plt.savefig("/home/cz136/project/balrog/{}.png".format(i),dpi=500)
    plt.show()

# plt.show()

In [160]:
chi_sqr={}
for key in keys:
    chi_sqr[key]=get_chisqr_for_key(key)

In [152]:
print(chi_sqr)

{'ebv_planck13': nan}
